# Delta Lake Lab 
## Unit 7: ZORDER & DATA SKIPPING

In the previous unit, we -
1. Learned how to time travel

In this unit, we will-
1. Learn about Z-Ordering and how it further optimizes data skipping

Z-Ordering is a (multi-dimensional clustering) technique to colocate related information in the same set of files. This co-locality is automatically used by Delta Lake in data-skipping algorithms. This behavior dramatically reduces the amount of data that Delta Lake on Apache Spark needs to read. To Z-Order data, you specify the columns to order on in the ZORDER BY clause.

Data skipping information is collected automatically when you write data into a Delta Lake table. Delta Lake takes advantage of this information (minimum and maximum values for each column) at query time to provide faster queries. You do not need to configure data skipping; the feature is activated whenever applicable. However, its effectiveness depends on the layout of your data. For best results, apply Z-Ordering.

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

23/12/02 23:54:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-diy-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-diy-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  11002190840


In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

gs://dll-data-bucket-11002190840/delta-consumable


In [7]:
# Lets take a look at the data lake before the zordering 
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part*

gs://dll-data-bucket-11002190840/delta-consumable/part-00000-18ffd7b0-964d-4f1e-a648-0892a1d4373d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5afc31da-4141-4f21-93e5-0ee8635aee9f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-5f4a2030-2b43-4229-815a-55fca555d323-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-687d9651-589c-4d6c-a8fd-df5d01770e85-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6dbd9c8a-28bb-48a7-b1a9-5304b8f16bc2-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-0

The author's output was-
```
gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-228eb829-1144-4a6d-a0e2-4fd39d9e9f57-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-29d7051f-ea28-4bd9-a8fd-8d9f8e38f163-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-3eb98e04-4353-4e5f-a8a2-17a570111981-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-47c79787-64f2-453c-8474-52cbcaeef3c2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-48902d8f-0572-4735-b0d7-95b1927cb294-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-598882f5-6a6b-489f-a551-6e782e67702f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-61eebddf-adb1-4088-9520-6a953e6d3fff-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-7b773887-eac1-481b-aa71-e57bd469f977-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-89dfe8e5-2d40-49f6-b0a0-4b320db62d14-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b5620589-f197-4417-8fcb-ce013d82deb9-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c36f0527-e995-41d2-a8ea-776cc865f816-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-487147d6-23f9-4a07-ae73-7cddbe8dfd06-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-8d9a7ec0-3a4d-43e4-8e33-379ffb2e4a3a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-91311981-1505-4747-a8c9-6b3616efb4c0-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00001-ad3ddbfe-cec8-4877-8b05-3d40da1079ba-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-15ea05a6-24a6-41e3-b9d1-7073429eda5f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-1717bbe6-6038-404c-aea4-e83d93eb0fa8-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00002-d43dcdd6-5fb8-4b31-baa2-5458bf321285-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-4c2e181f-c1eb-458f-aa08-9be435e56bb5-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-c15a6560-6d08-41f9-9ff8-546a02d4fca7-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00003-e6f811e0-d076-4619-bb1f-4e1ffd7caeb0-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00004-7663ab3e-1fbb-4ef9-bb08-80d4e6e3d9f4-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00005-c9b6ff00-72cb-4264-8f2a-d01f6c27b759-c000.snappy.parquet
```

In [8]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

37


### 4. ZORDER

In [9]:
spark.sql("OPTIMIZE loan_db.loans_by_state_delta ZORDER BY (addr_state)").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used
23/12/02 23:55:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                             |metrics                                                                                                                                                                                          |
+-------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|gs://dll-data-bucket-11002190840/delta-consumable|{1, 32, {7688, 7688, 7688.0, 1, 7688}, {993, 2048, 1539.625, 32, 49268}, 1, {all, {0, 0}, {32, 49268}, 0, {32, 49268}, 1, null}, 1, 32, 0, false, 0, 0, 1701561321279, 0, 8, 0, null, null, 3, 3}|
+---------------

In [10]:
# Lets take a look at the data lake post the zordering. There is one extra file, that appears to be a file that has all the data in it.  
!gsutil ls -lh $DELTA_LAKE_DIR_ROOT/part* | sort

     2 KiB  2023-12-02T23:51:00Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00007-263abe71-24ef-4a19-aef6-87fe728c9652-c000.snappy.parquet
     397 B  2023-12-02T23:40:33Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
     725 B  2023-12-02T23:39:48Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet
     725 B  2023-12-02T23:40:08Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00000-af39e13d-1030-4881-965c-406884eb9420-c000.snappy.parquet
     973 B  2023-12-02T23:39:28Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
     978 B  2023-12-02T23:19:54Z  gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet
     993 B  2023-12-02T23:40:40Z  gs://dll-data-bucket-11002190840/delta-consumable/part

In [11]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

38


In [12]:
# Lets take a look at the transaction log post the zordering 
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/:
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000003.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000004.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000005.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000006.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000007.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000008.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000009.json
gs://

In [13]:
# And review what is in the delta log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000011.json

{"commitInfo":{"timestamp":1701561060973,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":10,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"8","numOutputRows":"1632","numOutputBytes":"14163"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"5498c9c4-eeb1-4e4f-94f6-8f8d35ea1bc1"}}
{"add":{"path":"part-00000-5afc31da-4141-4f21-93e5-0ee8635aee9f-c000.snappy.parquet","partitionValues":{},"size":1717,"modificationTime":1701561060788,"dataChange":true,"stats":"{\"numRecords\":204,\"minValues\":{\"addr_state\":\"AK\",\"count\":0,\"collateral_value\":0.0},\"maxValues\":{\"addr_state\":\"WY\",\"count\":8942,\"collateral_value\":8.94202385061306E7},\"nullCount\":{\"addr_state\":0,\"count\":0,\"collateral_value\":0}}"}}
{"add":{"path":"part-00001-af96fa63-d091-4bc6-9272-19d70a65e1f4-c000.snappy.parquet","partitionValues":{},"size":1742,"modificationTime":1701561060786,"dataChange":true,"stats":"{\"numRec

### THIS CONCLUDES THIS LAB. PROCEED TO THE NEXT NOTEBOOK.